# Load model visualisation server

In [ ]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.modules import ChartModule
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.modules import TextElement
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%run Overtourism.ipynb

def agent_portrayal(my_agent):
    return my_agent.get_portrayal()

# Create canvas grid
grid = CanvasGrid(agent_portrayal,
                  Constants.GRID_SIZE,
                  Constants.GRID_SIZE,
                  Constants.CANVAS_SIZE,
                  Constants.CANVAS_SIZE)

model_params = {
    'n_locals' : UserSettableParameter('slider', 'Locals', 50, 0, 100, 10),
    'pro_tourism_probability' : UserSettableParameter('slider', 'Pro tourism probability', 0.2, 0.0, 1.0, 0.1),
    'n_initial_tourists' : UserSettableParameter('slider', 'Tourists', 10, 0, 200, 10),
    'grow_n_tourists' : UserSettableParameter('checkbox', 'New tourists?', value=True),
    'agent_portrayal' : UserSettableParameter('choice', 'Agent portrayal', value='utility', choices=['utility', 'random']),
    'collect_data' : UserSettableParameter('checkbox', 'Collect data', value=True),
    'policy' : UserSettableParameter('choice', 'Tourism policy', value='none', choices=['none', 'tax', 'travel_limit']),
    'tax_rate' : UserSettableParameter('slider', 'Tax rate', 0.3, 0.0, 1.0, 0.1),
    'travel_limit' : UserSettableParameter('slider', 'Max tourists', 50, 0, 200, 10),
    'utility_weight_pleasure' : UserSettableParameter('slider', 'WEIGHT: Pleasure', 1.0, 0, 5, 0.1),
    'utility_weight_overcrowding' : UserSettableParameter('slider', 'WEIGHT: Overcrowding', 1.0, 0, 5, 0.1),
    'utility_weight_overcapacity' : UserSettableParameter('slider', 'WEIGHT: Overcapacity', 1.0, 0, 5, 0.1),
    'utility_weight_pollution' : UserSettableParameter('slider', 'WEIGHT: Pollution', 1.0, 0, 5, 0.1),
    'utility_weight_wealth_change' : UserSettableParameter('slider', 'WEIGHT: Wealth change', 1.0, 0, 5, 0.1),
    'utility_weight_mood' : UserSettableParameter('slider', 'WEIGHT: Mood', 1.0, 0, 5, 0.1),
}

# Create chart

chart_town_wealth = ChartModule(
    [
        {'Label': 'town_wealth', 'Color': 'Orange'}
    ],
    data_collector_name='datacollector')

chart_utility = ChartModule(
    [
        {'Label': 'mean_local_utility', 'Color': 'Blue'},
        {'Label': 'mean_tourist_utility', 'Color': 'Red'},
        {'Label': 'mean_tourist_utility_hist', 'Color': 'Green'},
    ],
    data_collector_name='datacollector')

local_factors = {
    'overcapacity' : 'Black',
    'overcrowding' : 'Red',
    'pollution' : 'Blue',
    'wealth_change' : 'Orange',
    'pleasure' : 'Purple'
}
chart_local_factors = []
for local_type in ['lpt', 'lnpt']:
    factors = []
    for factor, color in local_factors.items():
        factors.append({'Label': 'mean_{}_{}'.format(factor, local_type), 'Color': color})
    chart_local_factors.append(ChartModule(factors, data_collector_name='datacollector'))

chart_factors_tourist = ChartModule(
    [
        {'Label': 'mean_overcapacity_t', 'Color': 'Black'},
        {'Label': 'mean_overcrowding_t', 'Color': 'Red'},
        {'Label': 'mean_pollution_t', 'Color': 'Blue'},
        {'Label': 'mean_mood_t', 'Color': 'Green'},
        {'Label': 'mean_pleasure_t', 'Color': 'Purple'},
    ],
    data_collector_name='datacollector')

chart_n_tourists = ChartModule(
    [
        {'Label': 'n_tourists', 'Color': 'Black'},
    ],
    data_collector_name='datacollector')

charts = [chart_utility, *chart_local_factors, chart_factors_tourist, chart_n_tourists]

class MyTextElement(TextElement):
    def render(self, model):
        monitors = {
            'Number of tourists' : 'n_tourists',
            'Avg. local utility' : 'mean_local_utility',
            'Avg. tourist utility' : 'mean_tourist_utility',
            'Avg. tourist utility (historical)' : 'mean_tourist_utility_hist',
            'FACTOR [Locals - pro tourism]: Pollution' : 'mean_pollution_lpt',
            'FACTOR [Locals - not pro tourism]: Pollution' : 'mean_pollution_lnpt',
            'FACTOR [Tourists]: Pollution' : 'mean_pollution_t',
            'FACTOR [Locals - pro tourism]: Overcrowding' : 'mean_overcrowding_lpt',
            'FACTOR [Locals - not pro tourism]: Overcrowding' : 'mean_overcrowding_lnpt',
            'FACTOR [Tourists]: Overcrowding' : 'mean_overcrowding_t',
            'FACTOR [Locals - pro tourism]: Overcapacity' : 'mean_overcapacity_lpt',
            'FACTOR [Locals - not pro tourism]: Overcapacity' : 'mean_overcapacity_lnpt',
            'FACTOR [Tourists]: Overcapacity' : 'mean_overcapacity_t',
            'FACTOR [Locals - pro tourism]: Pleasure' : 'mean_pleasure_lpt',
            'FACTOR [Locals - not pro tourism]: Pleasure' : 'mean_pleasure_lnpt',
            'FACTOR [Tourists]: Pleasure' : 'mean_pleasure_t',
            'FACTOR [Locals - not pro tourism]: Wealth change' : 'mean_wealth_change_lpt',
            'FACTOR [Locals - pro tourism]: Wealth change' : 'mean_wealth_change_lnpt',
            'FACTOR [Tourists]: Friendliness/Hostility' : 'mean_mood_t',
            'Town wealth' : 'town_wealth'
        }
        items = ['{}: {:.2f}'.format(k, float(model.datacollector.model_vars[v][-1]) if model.datacollector.model_vars[v] else -1) for k, v in monitors.items()]
        output = '<br>'.join(items)
        output += '<br>' + (model.datacollector.model_vars['debug'][-1] if model.datacollector.model_vars['debug'] else '-')
        return output

# Create and launch server

server = ModularServer(OvertourismModel,
                       [grid, MyTextElement()],#, chart_utility, *chart_local_factors, chart_factors_tourist, chart_n_tourists],
                       'Overtourism model',
                       model_params)
server.launch(port=1234)

# Reload model

In [ ]:
"""Reload model"""
# Run this cell if you've made changes to the model
# and want to test your changes without restarting the server
%run Overtourism.ipynb
server.model_cls = OvertourismModel
server.reset_model()

# Plot chart

In [ ]:
"""Plot current model run"""
def plot(chart):
    dfs = []
    model_report = server.model.datacollector.get_model_vars_dataframe()
    for item in chart.series:
        df = pd.DataFrame()
        label = item['Label']
        df['tick'] = model_report.index
        df['value'] = model_report[label]
        df['key'] = label
        dfs.append(df)
    df = pd.concat(dfs)
    keys_sorted = df.key.unique()
    keys_sorted.sort()
    
    sns.set(style='whitegrid', font_scale=1.0)
    fig, ax = plt.subplots(1, 1, figsize=(12,4))

    ax = sns.lineplot(
        data=df,
        x='tick',
        y='value',
        ax=ax,
        palette='Set1',
        hue='key',
        hue_order=keys_sorted,
        style='key',
        dashes=False)

    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles[1:], labels=labels[1:], loc='lower right')
    ax.set(xlabel='tick',
           ylabel='value')
    ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

for chart in charts:
    plot(chart)

# Kill server process

In [ ]:
"""Bash command to kill process"""
# Run the following process in Bash to kill the server process
# kill $(lsof -t -i:1234)

# Analysis

In [ ]:
# pro tourism
base_dir = '~/Downloads/'
result_csvs = {'Satisfaction Utility : 70% pro-tourism' : ['model_run_7_3.csv', 'model_run.csv', 'model_run_1.csv'],
               'Satisfaction Utility : 30% pro-tourism' : ['model_run_3_7.csv', 'model_run2.csv', 'model_run_2.csv'],
              }
columns_2 = [[('mean_local_pt_utility', 'Locals (Pro-tourism)'),
              ('mean_local_npt_utility', 'Locals (Not pro-tourism)'),
              ('mean_local_utility', 'Locals')],
             [('mean_tourist_utility', 'Tourists')]
            ]
policies = ['none', 'travel_limit', 'tax']
for param, csvs in result_csvs.items():
    for columns in columns_2:
        dfs = []
        for csv in csvs:
            result_df = pd.read_csv(base_dir + csv)
            for column in columns:
                df = pd.DataFrame()
                df['policy'] = result_df['policy']
                df['column'] = column[1]
                df['value'] = result_df[column[0]]
                dfs.append(df)
        if dfs:
            df = pd.concat(dfs)
            sns.set(style='whitegrid', font_scale=2.0)
            plt.figure(figsize=(16, 12))
            
            ax = sns.boxplot(
                data=df,
                x='column',
                y='value',
                palette='Set1',
                hue='policy',
                hue_order=policies,
                linewidth=1.5,
                showfliers = False,
            )
            handles, labels = ax.get_legend_handles_labels()
            ax.legend(handles=handles[1:], labels=labels[1:], loc='lower right')
            ax.set(xlabel=param,
                   ylabel='value')
            ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)